In [11]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://dim.kln.ac.lk/index.php/staff/academic-staff'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

urls = []
profile_buttons = soup.find_all('a', class_='sppb-btn-custom')

for button in profile_buttons:
    profile_url = button.get('href')
    if profile_url:
        urls.append(profile_url)

ac_staff = []

for url in urls:
    response = requests.get(url)
    response = response.content
    
    html_code = response.decode('utf-8')
    soup = BeautifulSoup(html_code, 'html.parser')
    
    # Extracting the name
    name = soup.find(class_='sppb-person-name').get_text()
    
    # Extracting the room using a pattern
    room_pattern = re.compile(r"Room\s*:\s*([^\n\r]*)")
    room_match = room_pattern.search(html_code)
    room = room_match.group(1).strip() if room_match else ''
    room = room.split('<')[0].strip()
    
    # Extracting the phone using a pattern
    phone_pattern = re.compile(r"Phone\s*:\s*([^\n\r]*)")
    phone_match = phone_pattern.search(html_code)
    phone = phone_match.group(1).strip() if phone_match else ''
    phone = phone.split('<')[0].strip()

    # Extracting the email 
    email_element = soup.find('a', href=lambda href: href and 'mailto' in href)
    email = email_element.text if email_element else ''

    # Extracting the specilization area 
    specialization_divs = soup.find_all('div', class_='sppb-panel sppb-panel-modern')
        
    if specialization_divs:
        for specialization_div in specialization_divs:
            if specialization_div.find(string=lambda text: text and 'Specialization' in text):               
                li_tags = specialization_div.find_all('li')
                specializations = []
                for li_tag in li_tags:
                    special = li_tag.text.strip()
                    specializations.append(special)

    sparea = ', '.join(specializations)
    
    ac_staff.append([name, room, phone, email,sparea]) 
    
# Creating a DataFrame
df = pd.DataFrame(ac_staff, columns=['Name', 'Room', 'Contact', 'Email','Specialization Areas'])

for staff_member in ac_staff:
    print(staff_member)

df.to_csv('Staff Table.csv', index=False)

['Prof. Janaka Wijayanayake', 'A4 201', '+94 (0)11 2914482(ext204)', 'janaka@kln.ac.lk', 'Management Information Systems']
['Dr. Suren Peter', 'A4.203', '+94 (0)11 2914482(ext203)', 'suren@kln.ac.lk', 'Finance']
['Prof. Darshana Sedera', '', '', 'darshana.sedera@scu.edu.au', '']
['Dr. Ruwan Wickramarachchi', 'A4 203', '+94 (0)11 2914482(ext206)', 'ruwan@kln.ac.lk', 'Management Information Systems, Electronic Commerce, Operations Management']
['Dr. Shantha jayalal', 'A4 302', '+94 (0)11 2914482(ext107)', 'shantha@kln.ac.lk', 'Computer Science']
['Dr. Thareendhra Keerthi Wijayasiriwardhane', 'A4.302', '+94 (0)11 2914482(ext108)', 'thareen@kln.ac.lk', 'Systems and Software Engineering']
['Dr. Dilani Wickramaarachchi', 'A4.201', '+94(0)11 2914482(ext207)', 'dilani@kln.ac.lk', 'Software Engineering']
['Dr. Chathura Rajapakse', 'A4.302', '+94 (0)11 2914482(ext103)', 'chathura@kln.ac.lk', 'Computational Intelligence and Systems Science']
['Dr. Amila Withanaarachchi', 'A4 203', '+94 (0)11 2914